In [1]:
import csv

from pathlib import Path

from owlready2 import *

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [18]:
# set up out path
data_p = Path("../../ontologies")

out_p = data_p / "out/"
out_p.mkdir(exist_ok=True)

node_p = out_p / "nodes.csv"
with open(node_p, 'w') as csvfile:
    fieldnames = [':ID', 'descriptive_label:string[]', 'iri', ':LABEL']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

rel_p  = out_p / "rel.csv"
with open(rel_p, 'w') as csvfile:
    fieldnames = [':START_ID', ':END_ID', ':TYPE', 'restriction']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

In [15]:
def create_transitionary_node(node_type, node_counter):
    # pass
    # AND_00000001
    # OR_00000001
    # BLANK_00000001
    node_counter = node_counter + 1
    node = f"{node_type.upper()}_{node_counter}"
    return node, node_counter

def append_node(node, node_p:Path, *, node_type:str=''):
    '''
    input:
        node:
            case 1: owlready2.entity.ThingClass
            case 2: Type: str: node = string name
        node_p: path to node file to append node to
        onto_type: node type for and/or
    '''
    # construct node row
    if isinstance(node, owlready2.entity.ThingClass):
        # ThingClass node
        node_line = [str(node.name), ';'.join(node.label), str(node.iri), node_type]
    elif isinstance(node, str):
        # Case: blank, and/or nodes
        node_line = [node, '', '', node_type]
    else:
        raise ValueError
    # write node to file
    with open(node_p, 'a') as f:
        node_writer = csv.writer(f, delimiter=',')
        node_writer.writerow(node_line)
        
def append_relation(source_id:str, target_id:str,
                    edge_type:str, *, restriction:str=''):
    # construct relation row
    rel_line = [source_id, target_id, edge_type, restriction]
    # write to file
    with open(rel_p, 'a') as f:
        rel_writer = csv.writer(f, delimiter=',')
        rel_writer.writerow(rel_line)

In [ ]:
# skeleton loop
raise NotImplementedError
and_counter = 0
or_counter = 0
blank_counter = 0
for c in onto.classes():
    # Append class node to class csv
    try:
        # check if label is present
        c.label
        
        # add class to node file
        
        # check if equivalence is not empty
        if list(c.equivalent_to):
            for sc in c.equivalent_to:
                parse_axiom(sc)
            # parse equivalence and continue
            # parse_grammar(parent_node, relation_type)
            # parse_grammar(c, "equivalent_to")
        elif list(c.is_a):
            for sc in c.is_a:
                parse_axiom(sc)
            # parse is_a
            # parse_grammar()
            # parse_grammar(c, "is_a")
        else:
            # log node with neither
    except ValueError:
        #TODO write to log file
        # parse grammar, leave descriptive label as empty for now
        # pass

obo.CDNO_0000001


In [4]:
# set paths
ontology_p = "../../ontologies"

foodon_p   = ontology_p + "/foodon_filtered.owl"

In [5]:
# load ontology
onto = get_ontology(foodon_p).load()

In [37]:
# get types
rel_types = set()
for c in onto.classes():
    # if c == "obo.FOODON_03420103":
    #     break
    # break
    if list(c.equivalent_to):
        for sc in c.equivalent_to:
            rel_types.add(type(sc))
            if isinstance(sc, owlready2.class_construct.Or):
                print(c.iri)
    elif list(c.is_a):
        for sc in c.is_a:
            rel_types.add(type(sc))
            if isinstance(sc, owlready2.class_construct.Or):
                print(c.iri)
rel_types

http://purl.obolibrary.org/obo/FOODON_03420103
http://purl.obolibrary.org/obo/FOODON_03460263
http://purl.obolibrary.org/obo/FOODON_03411563
http://purl.obolibrary.org/obo/FOODON_03460177
http://purl.obolibrary.org/obo/FOODON_03460231
http://purl.obolibrary.org/obo/FOODON_03310788
http://purl.obolibrary.org/obo/FOODON_03440022
http://purl.obolibrary.org/obo/FOODON_00002200
http://purl.obolibrary.org/obo/FOODON_03411021
http://purl.obolibrary.org/obo/FOODON_03490194
http://purl.obolibrary.org/obo/FOODON_03315308
http://purl.obolibrary.org/obo/FOODON_03412628
http://purl.obolibrary.org/obo/FOODON_03000217
http://purl.obolibrary.org/obo/FOODON_03000218
http://purl.obolibrary.org/obo/FOODON_03000219
http://purl.obolibrary.org/obo/FOODON_03000220
http://purl.obolibrary.org/obo/FOODON_03000225
http://purl.obolibrary.org/obo/FOODON_03000226
http://purl.obolibrary.org/obo/FOODON_03000227
http://purl.obolibrary.org/obo/FOODON_03000228
http://purl.obolibrary.org/obo/FOODON_03000229
http://purl.o

{owlready2.class_construct.And,
 owlready2.class_construct.Or,
 owlready2.class_construct.Restriction,
 owlready2.entity.ThingClass}

In [31]:
c

.time:TemporalUnit

In [7]:
for c in onto.classes():
    # if c.name == "FOODON_00002114":
    #     break
    break
print(c.iri)
print(f"object: {c} label: {c.label}")
print(f"equivalent to: {c.equivalent_to}")
print(f"is a: {c.is_a}")

http://purl.obolibrary.org/obo/FOODON_00002403
object: obo.FOODON_00002403 label: ['food material']
equivalent to: [obo.ENVO_00010483 & obo.RO_0000087.some(obo.CHEBI_33290)]
is a: [obo.ENVO_00010483, obo.RO_0000087.some(obo.CHEBI_33290)]


In [8]:
c.equivalent_to

[obo.ENVO_00010483 & obo.RO_0000087.some(obo.CHEBI_33290)]

In [ ]:
def parse_logic(source, target, edge_type, *, restriction=''):
    # raise NotImplementedError
    # TODO figure out inputs and outputs
    # Plan: execute recurssively until tyoe ThingClass found
    match type(source):
        case owlready2.entity.ThingClass:
            append_rel()
            # case: stop rule
            # create edge
            # return
        case owlready2.class_construct.And:
            # make recursive call to drill down further -> (blank_and, relation_type)
            trans_node = create_transitionary_node(node_type='and', target_node)
            append_node(trans_node)
            append_rel() # append relations from target to AND node
        case owlready2.class_construct.Or:
            # make recursive call to parse_logic -> (blank_or, relation_type)
            create_transitionary_node(node_type='or', target_node)
            append_rel() # append relations from target to OR node
            append_node(trans_node)
        case owlready2.class_construct.Restriction:
            # make recursive call to create edge
            create_transitionary_node('BLANK', target_node)
            append_node(trans_node)
            append_rel() # append relations from target to BLANK node
            parse_logic(trans_node, target.value, target.property)
            assert "some" in str(source)
            # add assert to verify only some restriction
        case _:
            # print out type
            raise TypeError
    return sub_axioms

In [38]:
def append_node(onto_class, node_p, *, onto_type=''):
    # construct node row
    node_line = [str(onto_class.name), str(onto_class.label), str(onto_class.iri), str(onto_type)]
    # write node to file
    with open(node_p, 'a') as f:
        node_writer = csv.writer(f, delimiter=',')
        node_writer.writerow(node_line)
        
def append_relation(parent:owlready2.entity.ThingClass, target:owlready2.entity.ThingClass,
                    edge_type:str, *, restriction:str=''):
    # construct relation row
    rel_line = [str(parent.name), str(target.name), edge_type, restriction]
    # write to file
    with open(rel_p, 'a') as f:
        rel_writer = csv.writer(f, delimiter=',')
        rel_writer.writerow(rel_line)

In [19]:
append_node("or_0001", node_p, node_type='or')
append_node("and_0001", node_p, node_type="and")
append_node(c, node_p)
append_node(c, node_p)

In [47]:
append_relation(c, c, "test", restriction="some")
append_relation(c, c, "test", restriction="some")
append_relation(c, c, "test", restriction="some")

In [9]:
type(c.equivalent_to)

owlready2.entity._EquivalentToList

In [8]:
type(list(c.equivalent_to)[0])

owlready2.class_construct.And

In [8]:
#TODO need to verify against a class with different is_a and equivalent_to
c.equivalent_to[0].get_Classes() # same as doing is_a below
c.equivalent_to[0].get_is_a() # also same as doing is_a

[obo.ENVO_00010483, obo.RO_0000087.some(obo.CHEBI_33290)]

In [9]:
c.equivalent_to[0].is_a

[obo.ENVO_00010483, obo.RO_0000087.some(obo.CHEBI_33290)]

In [10]:
type(c.equivalent_to[0].is_a[0])

owlready2.entity.ThingClass

In [11]:
c.equivalent_to[0].is_a[-1]

obo.RO_0000087.some(obo.CHEBI_33290)

In [12]:
type(c.equivalent_to[0].is_a[-1]) # if type restriction

owlready2.class_construct.Restriction

In [13]:
c.equivalent_to[0].is_a[-1].property # grab property as edge

obo.RO_0000087

In [14]:
c.equivalent_to[0].is_a[-1].value # grab value as node

obo.CHEBI_33290

In [15]:
c.equivalent_to[0].is_a[-1].type #TODO: is this specific to some? if so, what do the other map to?

24

In [16]:
type(list(c.is_a)[0])

owlready2.entity.ThingClass

In [25]:
rel_list = c.is_a
# isinstance(rel_list[-1], owlready2.class_construct.And)
type(rel_list[-1])

owlready2.class_construct.And

In [39]:
len(list(c.equivalent_to))

0

In [10]:
for c in onto.classes():
    if c.name == "FOODON_00002114":
        break
print(c.iri)
print(f"object: {c} label: {c.label}")
print(f"equivalent to: {c.equivalent_to}")
print(f"is a: {c.is_a}")

http://purl.obolibrary.org/obo/FOODON_00002114
object: obo.FOODON_00002114 label: ['pie food product']
equivalent to: []
is a: [obo.FOODON_00001180, obo.FOODON_00002347, obo.FOODON_00002644, obo.BFO_0000051.some(obo.FOODON_03306766) & obo.BFO_0000051.some(obo.FOODON_03306773) & obo.RO_0000086.some(obo.FOODON_00002407)]


In [11]:
[type(sc) for sc in c.is_a]

[owlready2.entity.ThingClass,
 owlready2.entity.ThingClass,
 owlready2.entity.ThingClass,
 owlready2.class_construct.And]

In [14]:
c.is_a[-1]

obo.BFO_0000051.some(obo.FOODON_03306766) & obo.BFO_0000051.some(obo.FOODON_03306773) & obo.RO_0000086.some(obo.FOODON_00002407)

In [15]:
c.is_a[-1].is_a

[obo.BFO_0000051.some(obo.FOODON_03306766),
 obo.BFO_0000051.some(obo.FOODON_03306773),
 obo.RO_0000086.some(obo.FOODON_00002407)]

In [21]:
c.is_a[-1].is_a
[type(sc_restriction) for sc_restriction in c.is_a[-1].is_a]

[owlready2.class_construct.Restriction,
 owlready2.class_construct.Restriction,
 owlready2.class_construct.Restriction]